In [148]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import re
import pandas as pd

In [149]:
driver = webdriver.Edge()
driver.get("http://www.instagram.com")
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
username.clear()
username.send_keys("fatizohra.mlk")
password.clear()
password.send_keys("Fatima@2000")
button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))
button.click()
try:
    not_now_button = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Save Info")]')))
    not_now_button.click()
except Exception as e:
    pass
try:
    not_now_button = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]')))
    not_now_button.click()
except Exception as e:
    pass
time.sleep(5)  
keyword = "hespress"
driver.get(f"https://www.instagram.com/{keyword}/reels/")
time.sleep(5)  

In [150]:
n_scrolls = 10
for j in range(0, n_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

In [151]:
# Obtenir tous les liens 'a'
liens = driver.find_elements(By.TAG_NAME,'a')

# Filtrer les liens commençant par "https://www.instagram.com/reel/"
video_links = [a.get_attribute('href') for a in liens if str(a.get_attribute('href')).startswith("https://www.instagram.com/reel/")]

# Imprimer les résultats
print('Found ' + str(len(video_links)) + ' video links')
print(video_links[:5])

Found 24 video links
['https://www.instagram.com/reel/Czyxee-rxoT/', 'https://www.instagram.com/reel/CzysUg8qjZw/', 'https://www.instagram.com/reel/CzyoY4HIBgY/', 'https://www.instagram.com/reel/CzykVOlhw5e/', 'https://www.instagram.com/reel/CzydEIiMVV6/']


In [152]:
driver.get(video_links[0])

In [153]:
import html

In [161]:
#start_date = "2023-11-17"
#end_date = "2023-11-18"
def Statistiques(link, start_date, end_date):
    if not link.startswith('http'):
        print(f"Invalid URL: {link}")
        return None

    driver.get(link)
    page_title = driver.title
    comments_list = []
    time.sleep(10)
    
    # Trouver l'élément des likes
    likes_elements = driver.find_elements(By.CLASS_NAME, "html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs")
    
    if likes_elements:
        likes_element = likes_elements[0]
        likes_count = likes_element.text.replace('\u202f', '')
    else:
        likes_count = 0

    # Trouver les balises méta
    meta_tags = driver.find_elements(By.CSS_SELECTOR, 'meta[property="og:description"]')
    page_title = html.unescape(driver.title)

    if meta_tags:
        content = meta_tags[0].get_attribute("content")
        page_title = driver.title.replace('\u200f', '').replace('\xa0', '').replace('| Instagram', '')
          # Extraire le nombre de commentaires
        comments_match = re.search(r'(\d+) comments', content)
        comments_count = comments_match.group(1) if comments_match else None

        # Extraire la date
        time_element = driver.find_element(By.CLASS_NAME, "_aaqe")
        datetime = time_element.get_attribute("datetime")
        time_text = time_element.text  # Renommé en time_text
        Date = datetime.split('T')
        heure = Date[1]
        DATE = Date[0]
        #match = re.search(date_pattern, content)
        
        # Si une correspondance est trouvée, extraire la date correspondante
#         if match:
#             extracted_date = match.group(1)
#         else:
#             extracted_date = None
    else:
        # Ajuster les valeurs par défaut si les balises méta ne sont pas trouvées
        likes_count = 0
        comments_count = None
        extracted_date = None

    # Extraire la liste des commentaires
    comments = driver.find_elements(By.XPATH, '//span[@class="x1lliihq x1plvlek xryxfnj x1n2onr6 x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye xvs91rp xo1l8bm x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj"]')
    
    # Ajouter chaque commentaire à la liste des commentaires
    for comment in comments:
        comments_list.append(comment.text)
        
    duration_script = "return document.querySelector('video').duration;"
    duration = driver.execute_script(duration_script)
        
    if start_date <= DATE <= end_date:
            data = {
                'Page Title': page_title,
                'link': link,
                'Likes Count': likes_count,
                'Comments Count': comments_count,
                'Date': DATE,
                'heure': heure,
                'durée': duration,
                'Comments List': comments_list[1:-1]
            }
    else:
        return None
    return data

In [162]:
Statistiques(video_links[2], start_date, end_date)

{'Page Title': 'متظاهرون أمام البيت الأبيض يدعون بايدن لإيقاف إطلاق النار في غزة #هسبريس #Hespress #فيديو_هسبريس #احتجاج #البيت_الأبيض #إيقاف_إطلاق_النار…',
 'link': 'https://www.instagram.com/reel/CzyoY4HIBgY/',
 'Likes Count': '292',
 'Comments Count': '7',
 'Date': '2023-11-18',
 'heure': '14:41:53.000Z',
 'durée': "return document.querySelector('video').duration;",
 'Comments List': ['ok_bye_off_19',
  'Killed stop free Palestine',
  'mustapha_akhtale',
  'قدس نفسك فأنت لست فرصة ثانية ولست خطة إحتياطية ولا علاج مؤقت ......!\nلاتجعل نفسك الخيار الثاني لأحد كن الأول أو لاتكن ......!\n\nيرحلون جميعا وتبقى لك نفسك التي حطمتها من أجلهم.......!\nالضمير شيء نادر لاتجده في كل إنسان .....!',
  'ninafashion.casa',
  'ياربي ما هذه الوحشية و الهمجيه ياربي نجنا من الصهاينة اصحاب الضمائر الميتة هاذو ليسوا ببشر اللهم شتتهم و مزقهم كل ممزق',
  'salimcherradi99',
  'دعائي هو اللهم إرزق إخواننا في فلسطين الصمود والقوة في وجه الطغيان وإنصرهم اللهم نج المستضعفين من المؤمنين اللهم إنصر ضعفهم ورد إلينا 

In [156]:
def get_statistics_for_all_reels(links, start_date, end_date):
    all_statistics = []

    for link in links:
        statistics = Statistiques(link, start_date, end_date)
        if statistics:
            all_statistics.append(statistics)

    return all_statistics

In [157]:
all_reel_statistics = get_statistics_for_all_reels(video_links, "2023-11-17","2023-11-18")
all_reel_statistics

[{'Page Title': 'جراء القصف الإسرائيلي.. ما قصة الأسير الذي مات "هلعا" بين أيدي كتائب القسام في غزة؟ #أسير #كتائب_القسام #غزة',
  'link': 'https://www.instagram.com/reel/Czyxee-rxoT/',
  'Likes Count': '124',
  'Comments Count': '3',
  'Date': '2023-11-18',
  'heure': '16:01:11.000Z',
  'Comments List': ['hekayat.3mo',
   'عليهم من الله ما يستحقون',
   'adillouen',
   'جهنم وبئس المصير',
   'angel___rodriguez___190',
   'لا وقف لإطلاق النار ، دمروا حماس']},
 {'Page Title': 'احتجاج في منتدى “ميدايز” بطنجة ضد مداخلات أجنبية تدعم إسرائيل . #هسبريس #Hespress #فيديو_هسبريس #احتجاج #منتدى_ميدايز #طنجة #إسرائيل',
  'link': 'https://www.instagram.com/reel/CzysUg8qjZw/',
  'Likes Count': '5247',
  'Comments Count': '288',
  'Date': '2023-11-18',
  'heure': '15:16:24.000Z',
  'Comments List': ['___________zen____________',
   'المغربة باعو الصحراء على قيبال غزة؟؟ أصبار تسالي الحرب و إسرائيل تعود الترابي من جديد ل بوريطة😂😂',
   'aksil_amazigh_136',
   'انتم لستم عرب ، يجب عليكم أولا طرد الاحتلال 

In [158]:
df = pd.DataFrame(all_reel_statistics)

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel('resultats_reels.xlsx', index=False)